Save every 'n'th predicted images which contain PVs\
    - Much Faster, this generates JSON files, and txt(optional)

In [ ]:
import os
import json
import glob
import cv2
import numpy as np
from tqdm import tqdm
from ultralytics import YOLO
import time
import datetime

# ---------------------------
# Configuration
# ---------------------------
model = YOLO("/home/il72/cape_town_year_of_installation/YOLO_CapeTown_5K_single_categories(July_14)/runs/obb/pv_detection_5K_yolo11x_obb_single_class_lr0.0001/weights/best.pt")

image_dir = "/shared/data/climateplus2025/Prediction_for_poster_3_images_July21/CapeTown_Image_2023_tiles_1024_for_prediction"
image_paths = sorted(glob.glob(os.path.join(image_dir, "*.tif")))

output_img_dir = "/shared/data/climateplus2025/Prediction_for_poster_3_images_July21/Prediction/predicted_images"
output_json_dir = "/shared/data/climateplus2025/Prediction_for_poster_3_images_July21/Prediction/predicted_json"
output_txt_dir = "/shared/data/climateplus2025/Prediction_for_poster_3_images_July21/Prediction/predicted_txt"

os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_json_dir, exist_ok=True)
os.makedirs(output_txt_dir, exist_ok=True)

SAVE_EVERY_N = 1  # Save image every N
SAVE_TXT = True   # Save TXT for post-processing

# ---------------------------
# Start Runtime Timer
# ---------------------------
print("YOLO OBB inference started...")
start_time = time.time()

# ---------------------------
# Inference Loop
# ---------------------------
for image_path in tqdm(image_paths, desc="Running YOLO OBB inference"):

    image_name = os.path.basename(image_path)
    base_name = os.path.splitext(image_name)[0]

    try:
        tile_id = int(base_name.split("_")[-1])
    except ValueError:
        tile_id = hash(base_name) % 100000

    should_save_image = (tile_id % SAVE_EVERY_N == 0)

    results = model.predict(
        source=image_path,
        task="obb",
        conf=0.5,
        save=False
    )

    img = None
    if should_save_image:
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    json_result = {
        "image_name": image_name,
        "model_type": "YOLO_OBB",
        "num_predictions": 0,
        "predictions": [],
        "sampled_visualization_saved": False
    }

    prediction_counter = 1
    txt_lines = []

    for result in results:
        if not hasattr(result, "obb") or result.obb is None:
            continue

        names = result.names
        boxes = result.obb.xyxyxyxy.cpu().numpy()
        scores = result.obb.conf.cpu().numpy()
        classes = result.obb.cls.cpu().numpy().astype(int)

        height, width = img.shape[:2] if img is not None else (1, 1)

        for i in range(len(boxes)):
            coords = boxes[i].reshape(4, 2).astype(int)
            polygon_xy = [[int(pt[0]), int(pt[1])] for pt in coords]
            label = f"{names[classes[i]]} {scores[i]:.2f}"

            if should_save_image and img is not None:
                cv2.polylines(img, [coords], isClosed=True, color=(0, 0, 255), thickness=2)
                cv2.putText(img, label, tuple(coords[0]), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            pred_dict = {
                "prediction_id": f"{base_name}_pred_{prediction_counter:04d}",
                "class": names[classes[i]],
                "confidence": float(scores[i]),
                "polygon": polygon_xy
            }
            json_result["predictions"].append(pred_dict)
            prediction_counter += 1

            # Save to TXT format
            if SAVE_TXT:
                coords_norm = boxes[i].reshape(4, 2).astype(np.float32)
                coords_norm[:, 0] /= width
                coords_norm[:, 1] /= height

                txt_line = f"{classes[i]}"
                for x, y in coords_norm:
                    txt_line += f" {x:.6f} {y:.6f}"
                txt_line += f" {scores[i]:.6f}"
                txt_lines.append(txt_line)

    json_result["num_predictions"] = len(json_result["predictions"])

    if json_result["num_predictions"] > 0 and should_save_image and img is not None:
        output_image_path = os.path.join(output_img_dir, f"{base_name}_predicted.jpg")
        cv2.imwrite(output_image_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
        json_result["sampled_visualization_saved"] = True
        print(f"Saved predicted image: {output_image_path}")

    if json_result["num_predictions"] > 0:
        output_json_path = os.path.join(output_json_dir, f"{base_name}.json")
        with open(output_json_path, "w") as f:
            json.dump(json_result, f, indent=2)
        print(f"Saved JSON: {output_json_path}")

    if SAVE_TXT and len(txt_lines) > 0:
        output_txt_path = os.path.join(output_txt_dir, f"{base_name}.txt")
        with open(output_txt_path, "w") as f:
            f.write("\n".join(txt_lines) + "\n")
        print(f"Saved TXT: {output_txt_path}")
    elif SAVE_TXT:
        print(f"No detections, skipped TXT for: {image_name}")

# ---------------------------
# End Runtime Timer
# ---------------------------
end_time = time.time()
runtime_seconds = end_time - start_time
runtime_formatted = str(datetime.timedelta(seconds=int(runtime_seconds)))

print(f"\n Inference completed in {runtime_seconds:.2f} seconds ({runtime_formatted})")


YOLO OBB inference started...


Running YOLO OBB inference:   0%|          | 0/507 [00:00<?, ?it/s]


image 1/1 /shared/data/climateplus2025/Prediction_for_poster_July21/CapeTown_Image_2023_tiles_1024_for_prediction/2023_RGB_8cm_W24A_17_tile_0_0.tif: 1024x1024 (no detections), 40.2ms
Speed: 1.8ms preprocess, 40.2ms inference, 0.5ms postprocess per image at shape (1, 3, 1024, 1024)


Running YOLO OBB inference:   0%|          | 1/507 [00:00<04:05,  2.06it/s]

No detections, skipped TXT for: 2023_RGB_8cm_W24A_17_tile_0_0.tif

image 1/1 /shared/data/climateplus2025/Prediction_for_poster_July21/CapeTown_Image_2023_tiles_1024_for_prediction/2023_RGB_8cm_W24A_17_tile_0_1024.tif: 1024x1024 (no detections), 37.9ms
Speed: 1.9ms preprocess, 37.9ms inference, 0.5ms postprocess per image at shape (1, 3, 1024, 1024)
No detections, skipped TXT for: 2023_RGB_8cm_W24A_17_tile_0_1024.tif

image 1/1 /shared/data/climateplus2025/Prediction_for_poster_July21/CapeTown_Image_2023_tiles_1024_for_prediction/2023_RGB_8cm_W24A_17_tile_0_10240.tif: 1024x1024 None37.9ms
Speed: 1.8ms preprocess, 37.9ms inference, 1.8ms postprocess per image at shape (1, 3, 1024, 1024)


...

No detections, skipped TXT for: 2023_RGB_8cm_W57B_8_tile_9216_7168.tif

image 1/1 /shared/data/climateplus2025/Prediction_for_poster_July21/CapeTown_Image_2023_tiles_1024_for_prediction/2023_RGB_8cm_W57B_8_tile_9216_8192.tif: 1024x1024 (no detections), 39.8ms
Speed: 1.8ms preprocess, 39.8ms inference, 0.4ms postprocess per image at shape (1, 3, 1024, 1024)


Running YOLO OBB inference: 100%|█████████▉| 506/507 [01:01<00:00,  8.75it/s]

No detections, skipped TXT for: 2023_RGB_8cm_W57B_8_tile_9216_8192.tif

image 1/1 /shared/data/climateplus2025/Prediction_for_poster_July21/CapeTown_Image_2023_tiles_1024_for_prediction/2023_RGB_8cm_W57B_8_tile_9216_9216.tif: 1024x1024 (no detections), 39.8ms
Speed: 1.9ms preprocess, 39.8ms inference, 0.5ms postprocess per image at shape (1, 3, 1024, 1024)


Running YOLO OBB inference: 100%|██████████| 507/507 [01:01<00:00,  8.28it/s]

No detections, skipped TXT for: 2023_RGB_8cm_W57B_8_tile_9216_9216.tif

 Inference completed in 61.20 seconds (0:01:01)
